# Run QSVR experiments with multiple splits

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from qa_summer.QSVR import QSVR
from dimod import ExactSolver
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
from utils import nb_utils
from joblib import dump, load
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import datetime
from random import randint, random
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
#import neal #import to use simulated annealing sampler
#from dwave.system import DWaveSampler #import to select specific sampler

In [2]:
search_svr_hps = False
experiment_name = ''
#date = datetime.datetime.now().strftime("_%Y_%m_%d-%I:%M:%S.%f_%p")
#experiment_name = experiment_name + date

### Run experiments with fixed epsilon, C and gamma

In [ ]:
'''
WARNING: THIS CELL SENDS PROBLEMS TO D-WAVE MULTIPLE TIMES
REMEMBRER D-WAVE AVALIABLE TIME IS LIMITED
'''
if search_svr_hps: #ONLY RUNS FOR FIXED SVR PARAMS: epsilon, C and gamma
	nb_utils.exit_cell('Exiting cell as search_svr_hps is set to True')

# load data
df_info = nb_utils.get_df_info('mlpf')
df = pd.read_csv(df_info['df_path'])
df = df.drop(df[df.loss_99 == df.loss_99.max()].index)

# Select features
curve = nb_utils.get_curve(df_info=df_info, known_curve=0.25, df=df)
X = curve[:,[i for i in range(0,curve.shape[1],2)]]

# Prediction target
y = nb_utils.get_target(df_info,df)

# Scale data
x_scaler = QuantileTransformer(n_quantiles=50,random_state=0)
X = x_scaler.fit_transform(X)
y_scaler =  QuantileTransformer(n_quantiles=50,random_state=0)
y = y_scaler.fit_transform(y.reshape(-1, 1)).ravel()

rs = randint(0, 2**30)
num_runs = 10
r2 = np.zeros((num_runs, 7))
for i in range(num_runs):
	# train test split
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=rs+i)
	X_train = X_train[:79,:]
	y_train = y_train[:79]
	
	# QSVR code
	qsvr_model = QSVR.QSVR() # instantiate
	#RUN ON D-WAVE
	#set sampler
	#sampler =  DWaveSampler(region='na-west-1', solver='Advantage_system6.1')
	qsvr_model.fit(X_train, y_train, K = 3, B = 0.5, epsilon = 0.02, k0 = 0.005, xi=0.01, n_samples = 20, num_reads = 1000, random_seed=rs+i, n_samples_for_gamma_and_C_optimizations=40, gamma=0.1, C=67.61, use_custom_chainstrength=True, chain_mult=8)
	
	nb_utils.save_qsvr(qsvr_model, 'qsvr_attrs_'+experiment_name+'_rs'+str(rs)+'_i'+str(i)) # save QSVR for further predictions
	
	# evaluate QSVR
	y_pred = qsvr_model.predict(X_test)
	for j in range(7):
		r2[i,j] = r2_score(y_pred[j],y_test)
	print(f'Finished run {i} with r2 = {r2[i,:]} with mean = {r2[i,:].mean()}')

results = {
	'scores norm' : r2[:,0],
	'scores softmax' : r2[:,1],
	'scores lc norm' : r2[:,2],
	'scores lc softmax' : r2[:,3],
	'best set of alphas' : r2[:,4],
	'simple mean' : r2[:,5],
	'min energy' : r2[:,6]
}

dump(results, 'results_'+experiment_name+'_rs'+str(rs)+'.joblib')

for k in results.keys():
	print(f'{k}: {results[k]}')

### Run experiments searching for epsilon, C and gamma

In [3]:
'''
WARNING: THIS CELL SENDS PROBLEMS TO D-WAVE MULTIPLE TIMES
REMEMBRER D-WAVE AVALIABLE TIME IS LIMITED
'''
if not search_svr_hps: #ONLY RUNS if user wants to search for: epsilon, C and gamma
	nb_utils.exit_cell('Exiting cell as search_svr_hps is set to False')
	
# load data
df_info = nb_utils.get_df_info('mlpf')
df = pd.read_csv(df_info['df_path'])
df = df.drop(df[df.loss_99 == df.loss_99.max()].index)

# Select features
curve = nb_utils.get_curve(df_info=df_info, known_curve=0.25, df=df)
X = curve[:,[i for i in range(0,curve.shape[1],2)]]

# Prediction target
y = nb_utils.get_target(df_info,df)

# Scale data
x_scaler = QuantileTransformer(n_quantiles=50,random_state=0)
X = x_scaler.fit_transform(X)
y_scaler =  QuantileTransformer(n_quantiles=50,random_state=0)
y = y_scaler.fit_transform(y.reshape(-1, 1)).ravel()

rs = randint(0, 2**30)
num_runs = 5
r2 = np.zeros((num_runs, 6))
for i in range(num_runs):
	# train test split
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=200, random_state=rs+i)

	# Optimiza SVR hps using classical SVR
	best, C_best, epsilon_best, gamma_best = -100, None, None, None
	for j in range(5000):
		C = np.exp(np.random.uniform(np.log(1e-4),np.log(100.0)))
		gamma = 1/(X_train.shape[1]*X_train.var())
		epsilon = np.exp(np.random.uniform(np.log(1e-3),np.log(1)))
		model = SVR(C=C,epsilon=epsilon,gamma=gamma)
		cvs, __ = nb_utils.small_train_r2_cv(model, X=X_train, y=y_train, train_size= 20, reps=5, test_size=0.74,rs=rs+j)
		cvs = cvs.mean()
		if best < cvs:
			best = cvs
		C_best, epsilon_best, gamma_best = C, epsilon, gamma
        
	print(pd.DataFrame({'CV mean R^2': best, 'C': C_best, 'epsilon': epsilon_best, 'gamma': gamma_best}, index=['Best hyperparameters']))

	X_train, _, y_train, _ = train_test_split(X, y, train_size=20, random_state=rs+i)

	
	# QSVR code
	qsvr_model = QSVR.QSVR() # instantiate
	#RUN ON D-WAVE
	qsvr_model.fit(X_train, y_train, K = 3, B = 2, epsilon = epsilon_best, k0 = 0.005, xi=0.01, n_samples = 20, num_reads = 2500, random_seed=rs+i, n_samples_for_gamma_and_C_optimizations=0, gamma=gamma_best, C=C_best)
	nb_utils.save_qsvr(qsvr_model, 'qsvr_attrs_'+experiment_name+'_rs'+str(rs)+'_i'+str(i)) # save QSVR for further predictions
	# evaluate QSVR
	y_pred = qsvr_model.predict(X_test)
	for j in range(6):
		r2[i,j] = r2_score(y_pred[j],y_test)
	print(f'Finished run {i} with r2 = {r2[i,:]}')
results = {
	'scores norm' : r2[:,0],
	'scores softmax' : r2[:,1],
	'scores lc norm' : r2[:,2],
	'scores lc softmax' : r2[:,3],
	'best set of alphas' : r2[:,4],
	'simple mean' : r2[:,5]
}

dump(results, 'results_'+experiment_name+'_rs'+str(rs)+'.joblib')

for k in results.keys():
	print(f'{k}: {results[k]}')

Exiting cell as search_svr_hps is set to False


### Analize experiment results

In [ ]:
#results = load('results_B10_g01_k005_xi01_c67_rs'+str(rs)+'.joblib')
barWidth = 0.25
fig = plt.subplots(figsize =(12, 8))
plt.grid()
plt.gca().set_axisbelow(True)
 
# set height of bar
MEAN = [val.mean() for val in results.values()]
MEDIAN = [np.median(val) for val in results.values()]
MAX = [val.max() for val in results.values()]
 
# Set position of bar on X axis
br1 = np.arange(len(results))
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
 
# Make the plot
plt.bar(br1, MEAN, width = barWidth, label ='Mean')
plt.bar(br2, MEDIAN, width = barWidth, label ='Median')
plt.bar(br3, MAX, width = barWidth, label ='Max')
 
# Adding Xticks
plt.ylabel('R^2', fontweight ='bold', fontsize = 15)
plt.xticks([r + barWidth for r in range(len(results.keys()))], list(results.keys()))
 
plt.legend()
plt.show()